# **Pandas Tables**
## **01 Load DataSet**

In [15]:
import pandas
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
df = pandas.read_html(url, header=0)[0]
df['종목코드'] = df['종목코드'].map(lambda x: f'{x:0>6}')
df.head(2)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DL,000210,기타 금융업,지주회사,1976-02-02,12월,전병욱,http://www.dlholdings.co.kr,서울특별시
1,DRB동일,004840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시


In [16]:
# 필드 1개라도 NaN 포함 필터링
import numpy
print(df[df.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)].shape) 
df.isnull().sum()

(164, 9)


회사명       0
종목코드      0
업종        0
주요제품     18
상장일       0
결산월       0
대표자명      0
홈페이지    152
지역        0
dtype: int64

In [17]:
# 불필요한 필트 필터링
df.drop(labels=['주요제품', '홈페이지'], axis=1, inplace=True)
df.tail(2)

,회사명,종목코드,업종,상장일,결산월,대표자명,지역
2497,한국미라클피플사,331660,기타 화학제품 제조업,2019-10-28,12월,이호경,경기도
2498,휴벡셀,212310,의료용 기기 제조업,2016-07-26,12월,"박정민, 허성규(각자대표)",경기도


## **02 NaT Interpolation**
- NaT : `datetime` 의 `numpy.nan` 객체
- **Django ORM** 은`None` 사용  cf)<span style="color:red"> **numpy.nan** 은 `nan` 텍스트를 저장한다</span>

```python
# NaN 과 NaT 는 호환된다
pandas.to_datetime(df['상장일']) -> string to `datetime`
pandas.isnull(pandas.NaT) -> True, False
```

In [2]:
import numpy, random
for _ in range(2):  # 임의의 None 값 테이블에 추가하기
    df['상장일'][random.randint(1, len(df))] = numpy.nan
df['상장일'] = pandas.to_datetime(df['상장일'])
df[df['상장일'].isnull()]

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
57,윌비스,008600,생활용품 도매업,"스웨터,니트,셔츠 도매,제조/컴퓨터주변기기 무역,오파,임대",NaT,12월,"임찬혁(대표집행임원), 송주호(대표집행임원)",http://www.willbes.com,충청남도
1070,메드팩토,235980,자연과학 및 공학 연구개발업,항암제 신약개발,NaT,12월,김성진,http://www.medpacto.com,서울특별시


In [3]:
import datetime
index_list = df[df['상장일'].isnull()].index.tolist()

df['상장일'] = list(
    map(lambda x: 
        pandas.to_datetime(datetime.date(1900,1,1)).date() if (pandas.isnull(x)) 
        else x, df['상장일'] ))
df.loc[index_list,:]

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
57,윌비스,008600,생활용품 도매업,"스웨터,니트,셔츠 도매,제조/컴퓨터주변기기 무역,오파,임대",1900-01-01,12월,"임찬혁(대표집행임원), 송주호(대표집행임원)",http://www.willbes.com,충청남도
1070,메드팩토,235980,자연과학 및 공학 연구개발업,항암제 신약개발,1900-01-01,12월,김성진,http://www.medpacto.com,서울특별시


## **03 Filtering**
정규식을 사용한 필터링

In [4]:
token = "통신"
import re
tokenizer = re.compile(f'{token}')
tokens = list(filter(lambda x : len("".join(tokenizer.findall(x))) != 0 , df['업종'].to_list()))
", ".join(set(tokens))

'컴퓨터 및 통신장비 수리업, 전기 및 통신 공사업, 전기 통신업, 가전제품 및 정보통신장비 소매업, 통신 및 방송 장비 제조업'

In [5]:
df['업종'].to_list()[0]

'기타 금융업'